## Download Data

In [1]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2024-12-26 17:32:08--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip.1’

concrete_data_week4 100%[===================>] 249.37M  31.1MB/s    in 9.2s    

2024-12-26 17:32:18 (27.2 MB/s) - ‘concrete_data_week4.zip.1’ saved [261483817/261483817]



In [2]:
!unzip -q concrete_data_week4.zip

## Part 1

### design a classifier using the VGG16 pre-trained model

In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

In [4]:
num_classes = 2
batch_size = 100
image_size = (224, 224)

In [5]:
generator = ImageDataGenerator(preprocessing_function=preprocess_input)

training_generator = generator.flow_from_directory(
    "concrete_data_week4/train",
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
)
validation_generator = generator.flow_from_directory(
    "concrete_data_week4/valid",
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
)

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [6]:
model_vgg16 = Sequential()

model_vgg16.add(VGG16(include_top=False, pooling="avg", weights="imagenet",))
model_vgg16.add(Dense(num_classes, activation="softmax"))

model_vgg16.layers[0].trainable = False

model_vgg16.summary()

58889256/58889256 [==============================] - 3s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                                 
Total params: 14715714 (56.14 MB)
Trainable params: 1026 (4.01 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [7]:
model_vgg16.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)

In [8]:
num_epochs = 2
steps_per_epoch_training = len(training_generator)
steps_per_epoch_validation = len(validation_generator)

history_vgg16 = model_vgg16.fit_generator(
    training_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

/tmp/1000930000/ipykernel_525/1877265301.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_vgg16 = model_vgg16.fit_generator(


Epoch 1/2
301/301 [==============================] - 4467s 15s/step - loss: 0.1010 - accuracy: 0.9645 - val_loss: 0.0247 - val_accuracy: 0.9943
Epoch 2/2
301/301 [==============================] - 4406s 15s/step - loss: 0.0190 - accuracy: 0.9956 - val_loss: 0.0144 - val_accuracy: 0.9972


In [11]:
model_vgg16.save("classifier_vgg16_model.h5")

## Part 2

### Evaluate your deep learning models on a test data

In [17]:
from keras.models import load_model

model_resnet50 = load_model("classifier_resnet_model.h5")

In [19]:
testing_generator = generator.flow_from_directory(
    "concrete_data_week4/test", target_size=image_size, shuffle=False,
)

Found 500 images belonging to 2 classes.


In [20]:
performance_vgg16 = model_vgg16.evaluate_generator(testing_generator)
print("Performance of the VGG16-trained model")
print("Loss: {}".format(round(performance_vgg16[0], 5)))
print("Accuracy: {}".format(round(performance_vgg16[1], 5)))

/tmp/1000930000/ipykernel_525/3522363865.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  performance_vgg16 = model_vgg16.evaluate_generator(testing_generator)


Performance of the VGG16-trained model
Loss: 0.01083
Accuracy: 0.996


In [21]:
performance_resnet50 = model_resnet50.evaluate_generator(testing_generator)
print("Performance of the ResNet50-trained model")
print("Loss: {}".format(round(performance_resnet50[0], 5)))
print("Accuracy: {}".format(round(performance_resnet50[1], 5)))

/tmp/1000930000/ipykernel_525/2120773404.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  performance_resnet50 = model_resnet50.evaluate_generator(testing_generator)


Performance of the ResNet50-trained model
Loss: 0.00771
Accuracy: 0.996


## Part 3

### In this model, you will predict whether the images in the test data are images of cracked concrete or not

In [22]:
predictions_vgg16 = model_vgg16.predict_generator(testing_generator, steps=1)


def print_prediction(prediction):
    if prediction[0] > prediction[1]:
        print("Negative ({}% certainty)".format(round(prediction[0] * 100, 1)))
    elif prediction[1] > prediction[0]:
        print("Positive ({}% certainty)".format(round(prediction[1] * 100, 1)))
    else:
        print("Unsure (prediction split 50–50)")


print("First five predictions for the VGG16-trained model:")
for i in range(5):
    print_prediction(predictions_vgg16[i])

/tmp/1000930000/ipykernel_525/3533000246.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions_vgg16 = model_vgg16.predict_generator(testing_generator, steps=1)


First five predictions for the VGG16-trained model:
Negative (99.8% certainty)
Negative (97.5% certainty)
Negative (95.7% certainty)
Negative (99.5% certainty)
Negative (99.1% certainty)


In [23]:
predictions_resnet50 = model_resnet50.predict_generator(testing_generator, steps=1)
print("First five predictions for the ResNet50-trained model:")
for i in range(5):
    print_prediction(predictions_resnet50[i])

/tmp/1000930000/ipykernel_525/4152097660.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions_resnet50 = model_resnet50.predict_generator(testing_generator, steps=1)


First five predictions for the ResNet50-trained model:
Negative (100.0% certainty)
Negative (99.0% certainty)
Negative (98.3% certainty)
Negative (99.9% certainty)
Negative (99.8% certainty)
